In [1]:
import os
import pathlib
import json

import pandas as pd

In [2]:
ru_or_eng = 1

BASE_DIR = str(pathlib.Path().resolve())
TARGET_DIR = ''
WORKING_DIR = f'{BASE_DIR}/{TARGET_DIR}'

dir_list = os.listdir(WORKING_DIR)

mapping = ''

for file in dir_list:
    if file.endswith(".xlsx"):
        mapping = file

#
# print(f'BASE_DIR: {BASE_DIR}')
# print(f'TARGET_DIR: {TARGET_DIR}')
# print(f'WORKING_DIR: {WORKING_DIR}')
# print(f'dir_list: {dir_list}')
print(f'mapping: {mapping}')

mapping: S2T Arkhivatsiia IS SAS EG УБП v IS BAS v1.xlsx


In [3]:
df = pd.read_excel(f'{WORKING_DIR}/{mapping}', sheet_name='Mapping',
                   usecols="D,T:V,Z,AA")

df = df.drop(0,axis=0)

df.columns = ['SchemaS', 'SchemaT', 'Table', 'Code', 'Data Type', 'Length']

df = df[df['Code']!='hdp_processed_dttm']

df = df.sort_values(['Table'])

df.index = range(1, len(df) + 1)

df.head()

,SchemaS,SchemaT,Table,Code,Data Type,Length
1,dbo,prod_arch_saeg,accMark_1kv17,sgn,smallint,5
2,dbo,prod_arch_saeg,accMark_1kv17,cSLX,string,12
3,dbo,prod_arch_saeg,accMark_1kv17,cABS,string,50
4,dbo,prod_arch_saeg,accMark_1kv17,cID,int,10
5,dbo,prod_arch_saeg,accMark_1kv17,cur2,string,3


In [10]:
tables_lst = df['Table'].unique()
print(
    len(tables_lst)
)
print(tables_lst)

134
['accMark_1kv17' 'accMark_1kv18' 'accMark_1kv19' 'accMark_1kv20'
 'accMark_1kv21' 'accMark_2kv17' 'accMark_2kv18' 'accMark_2kv19'
 'accMark_2kv20' 'accMark_2kv21' 'accMark_3kv17' 'accMark_3kv18'
 'accMark_3kv19' 'accMark_3kv20' 'accMark_3kv21' 'accMark_4kv17'
 'accMark_4kv18' 'accMark_4kv19' 'accMark_4kv20' 'accMark_4kv21'
 'acc_1kv17' 'acc_1kv18' 'acc_1kv19' 'acc_1kv20' 'acc_1kv21' 'acc_2kv17'
 'acc_2kv18' 'acc_2kv19' 'acc_2kv20' 'acc_2kv21' 'acc_3kv17' 'acc_3kv18'
 'acc_3kv19' 'acc_3kv20' 'acc_3kv21' 'acc_4kv17' 'acc_4kv18' 'acc_4kv19'
 'acc_4kv20' 'acc_4kv21' 'bs_1kv17' 'bs_1kv18' 'bs_1kv19' 'bs_1kv20'
 'bs_1kv21' 'bs_2kv17' 'bs_2kv18' 'bs_2kv19' 'bs_2kv20' 'bs_2kv21'
 'bs_3kv17' 'bs_3kv18' 'bs_3kv19' 'bs_3kv20' 'bs_3kv21' 'bs_4kv17'
 'bs_4kv18' 'bs_4kv19' 'bs_4kv20' 'bs_4kv21' 'bsplAgr_1kv17'
 'bsplAgr_1kv18' 'bsplAgr_1kv19' 'bsplAgr_1kv20' 'bsplAgr_1kv21'
 'bsplAgr_2kv18' 'bsplAgr_2kv19' 'bsplAgr_2kv20' 'bsplAgr_2kv21'
 'bsplAgr_3kv17' 'bsplAgr_3kv18' 'bsplAgr_3kv19' 'bsplAgr_

In [5]:
test_df = df[df['Table']=='accMark_1kv17'].copy()
test_df.head()

,SchemaS,SchemaT,Table,Code,Data Type,Length
1,dbo,prod_arch_saeg,accMark_1kv17,sgn,smallint,5
2,dbo,prod_arch_saeg,accMark_1kv17,cSLX,string,12
3,dbo,prod_arch_saeg,accMark_1kv17,cABS,string,50
4,dbo,prod_arch_saeg,accMark_1kv17,cID,int,10
5,dbo,prod_arch_saeg,accMark_1kv17,cur2,string,3


In [6]:
test_row = test_df.iloc[0]
schema_s = test_row['SchemaS']
print(schema_s)
schema_t = test_row['SchemaT']
print(schema_t)
table = test_row['Table']
print(table)
columns = []
columns_casts = []

for _, row in test_df.iterrows():
    columns.append(row['Code'])
    columns_casts.append(
        {
            "name": row['Code'],
            "colType": row['Data Type']
        }
    )

print(columns)
print(columns_casts)

dbo
prod_arch_saeg
accMark_1kv17
['sgn', 'cSLX', 'cABS', 'cID', 'cur2', 'cur', 'do', 'gfi', 'gf', 'f6', 'f4', 'mod', 'bNum', 'bType', 'm', 'kv', 'cType', 'y', 'id', 'dep', 'nPL', 'hs', 'termGr', 'log', 'gr1', 'rowRule', 'mirGroup', 'mirCode', 'ctg', 'inc', 'urpzNew', 'urpz', 'branch', 'group', 'depM', 'stat']
[{'name': 'sgn', 'colType': 'smallint'}, {'name': 'cSLX', 'colType': 'string'}, {'name': 'cABS', 'colType': 'string'}, {'name': 'cID', 'colType': 'int'}, {'name': 'cur2', 'colType': 'string'}, {'name': 'cur', 'colType': 'string'}, {'name': 'do', 'colType': 'smallint'}, {'name': 'gfi', 'colType': 'smallint'}, {'name': 'gf', 'colType': 'smallint'}, {'name': 'f6', 'colType': 'string'}, {'name': 'f4', 'colType': 'string'}, {'name': 'mod', 'colType': 'smallint'}, {'name': 'bNum', 'colType': 'smallint'}, {'name': 'bType', 'colType': 'smallint'}, {'name': 'm', 'colType': 'smallint'}, {'name': 'kv', 'colType': 'smallint'}, {'name': 'cType', 'colType': 'string'}, {'name': 'y', 'colType': '

In [7]:
test_flow_entity_lst = []

template1 = {
    "loadType": "Scd1Replace",
    "source": {
        "schema": schema_s,
        "table": table,
        "columns": columns,
        "columnCasts": columns_casts,
        "jdbcDialect": "..."
    },
    "target": {
        "table": table
    }
}

test_flow_entity_lst.append(template1)

main_json_template = {
  "connection": {
    "connType": "jdbc",
    "url": "...",
    "driver": "...",
    "user": "...",
    "password": "..."
  },
  "commonInfo": {
    "targetSchema": schema_t,
    "etlSchema": schema_t,
    "logsTable": "logs597"
  },
  "flows": test_flow_entity_lst
}

json.dumps(main_json_template)

'{"connection": {"connType": "jdbc", "url": "...", "driver": "...", "user": "...", "password": "..."}, "commonInfo": {"targetSchema": "prod_arch_saeg", "etlSchema": "prod_arch_saeg", "logsTable": "logs597"}, "flows": [{"loadType": "Scd1Replace", "source": {"schema": "dbo", "table": "accMark_1kv17", "columns": ["sgn", "cSLX", "cABS", "cID", "cur2", "cur", "do", "gfi", "gf", "f6", "f4", "mod", "bNum", "bType", "m", "kv", "cType", "y", "id", "dep", "nPL", "hs", "termGr", "log", "gr1", "rowRule", "mirGroup", "mirCode", "ctg", "inc", "urpzNew", "urpz", "branch", "group", "depM", "stat"], "columnCasts": [{"name": "sgn", "colType": "smallint"}, {"name": "cSLX", "colType": "string"}, {"name": "cABS", "colType": "string"}, {"name": "cID", "colType": "int"}, {"name": "cur2", "colType": "string"}, {"name": "cur", "colType": "string"}, {"name": "do", "colType": "smallint"}, {"name": "gfi", "colType": "smallint"}, {"name": "gf", "colType": "smallint"}, {"name": "f6", "colType": "string"}, {"name": 

In [8]:
def print_flow_entity(schema_s, table, columns, columns_casts):
	template = {
		"loadType": "Scd1Replace",
		"source": {
			"schema": schema_s,
			"table": table,
			"columns": columns,
			"columnCasts": columns_casts,
			"jdbcDialect": "..."
		},
		"target": {
			"table": table
		}
	}

	return json.dumps(template)

json_str = print_flow_entity(schema_s, table, columns, columns_casts)

json_str

'{"loadType": "Scd1Replace", "source": {"schema": "dbo", "table": "accMark_1kv17", "columns": ["sgn", "cSLX", "cABS", "cID", "cur2", "cur", "do", "gfi", "gf", "f6", "f4", "mod", "bNum", "bType", "m", "kv", "cType", "y", "id", "dep", "nPL", "hs", "termGr", "log", "gr1", "rowRule", "mirGroup", "mirCode", "ctg", "inc", "urpzNew", "urpz", "branch", "group", "depM", "stat"], "columnCasts": [{"name": "sgn", "colType": "smallint"}, {"name": "cSLX", "colType": "string"}, {"name": "cABS", "colType": "string"}, {"name": "cID", "colType": "int"}, {"name": "cur2", "colType": "string"}, {"name": "cur", "colType": "string"}, {"name": "do", "colType": "smallint"}, {"name": "gfi", "colType": "smallint"}, {"name": "gf", "colType": "smallint"}, {"name": "f6", "colType": "string"}, {"name": "f4", "colType": "string"}, {"name": "mod", "colType": "smallint"}, {"name": "bNum", "colType": "smallint"}, {"name": "bType", "colType": "smallint"}, {"name": "m", "colType": "smallint"}, {"name": "kv", "colType": "s